In [ ]:
# ============ SETUP CELL - RUN FIRST ============
import os
import sys
import subprocess

# CONFIG: Change to False if running locally (e.g. Jupyter Lab)
RUN_ON_CLOUD = True

# --------------------------------------------------------------------------------
# 1. CLOUD PERSISTENCE (Google Drive)
# --------------------------------------------------------------------------------
if RUN_ON_CLOUD:
    try:
        from google.colab import drive

        print("💾 Mounting Google Drive...")
        drive.mount("/content/drive")
    except ImportError:
        print("⚠️ Not running on Google Colab or drive import failed.")
        RUN_ON_CLOUD = False

# --------------------------------------------------------------------------------
# 2. CLONE REPOSITORY
# --------------------------------------------------------------------------------
REPO_URL = "https://github.com/your_username/RL-NBV.git"
REPO_NAME = "RL-NBV"

if not os.path.exists(REPO_NAME):
    print("🚀 Cloning repository...")
    !git clone {REPO_URL}
    %cd {REPO_NAME}
else:
    print("📂 Repository already exists, entering directory...")
    %cd {REPO_NAME}
    !git pull origin master

# --------------------------------------------------------------------------------
# 3. SETUP PERSISTENT LINKS (Checkpoints, Logs, Results)
# --------------------------------------------------------------------------------
if RUN_ON_CLOUD and os.path.exists("/content/drive"):
    DRIVE_CHECKPOINT_PATH = "/content/drive/MyDrive/RL_NBV_Checkpoints"
    DRIVE_LOG_PATH = "/content/drive/MyDrive/RL_NBV_Logs"

    # Create folders in Drive if they don't exist
    os.makedirs(DRIVE_CHECKPOINT_PATH, exist_ok=True)
    os.makedirs(DRIVE_LOG_PATH, exist_ok=True)

    # List of directories to symlink (Local Name -> Drive Path)
    LINK_DIRS = {
        "checkpoints": DRIVE_CHECKPOINT_PATH,
        "runs": os.path.join(DRIVE_LOG_PATH, "tensorboard_runs"),  # Tensorboard logs
    }

    # List of files to symlink (Local Name -> Drive Path)
    LINK_FILES = {
        "train_detail.log": os.path.join(DRIVE_LOG_PATH, "train_detail.log"),
        "train_result.txt": os.path.join(DRIVE_LOG_PATH, "train_result.txt"),
    }

    print("🔗 Setting up persistent storage links...")

    # Handle Directories
    for local_name, drive_path in LINK_DIRS.items():
        if not os.path.exists(drive_path):
            os.makedirs(drive_path, exist_ok=True)
        if os.path.exists(local_name) and not os.path.islink(local_name):
            !rm -rf {local_name}  # Remove local folder to replace with link
        if not os.path.exists(local_name):
            !ln -s {drive_path} {local_name}
            print(f"   Directory Linked: {local_name} -> {drive_path}")

    # Handle Files
    for local_name, drive_path in LINK_FILES.items():
        # Ensure the file exists on Drive so symlink works (touch it)
        if not os.path.exists(drive_path):
            !touch {drive_path}
        if os.path.exists(local_name) and not os.path.islink(local_name):
            !rm -f {local_name}   # Remove local file to replace with link
        if not os.path.exists(local_name):
            !ln -s {drive_path} {local_name}
            print(f"   File Linked: {local_name} -> {drive_path}")
else:
    print("⚠️ Running locally or Drive not mounted. Persistence disabled.")

# --------------------------------------------------------------------------------
# 4. INSTALL DEPENDENCIES
# --------------------------------------------------------------------------------
print("📦 Installing dependencies...")
!pip install -r requirements.txt
!pip install pyyaml

# --------------------------------------------------------------------------------
# 5. BUILD C++ EXTENSIONS
# --------------------------------------------------------------------------------
print("🛠️ Building C++ extensions...")
subprocess.check_call([sys.executable, "setup.py", "build_ext", "--inplace"])

# --------------------------------------------------------------------------------
# 6. VERIFICATION
# --------------------------------------------------------------------------------
sys.path.insert(0, os.getcwd())
sys.path.insert(0, os.path.join(os.getcwd(), "distance"))

try:
    import torch
    import distance.chamfer_distance as cd

    print("✅ C++ Extensions imported successfully!")
    if torch.cuda.is_available():
        print(f"✅ CUDA available: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️  CUDA NOT FOUND - ENABLE GPU RUNTIME!")
except ImportError as e:
    print(f"❌ Build failed: {e}")

print("\n🎉 Environment Ready! Logs & Checkpoints are safe on Drive.")

In [ ]:
# ============ TRAIN CELL ============
# Run training using your config.yaml file
!python run_train.py --config config.yaml

In [ ]:
# ============ DRY RUN (OPTIONAL) ============
# Check commands without running them
!python run_train.py --config config.yaml --dry_run